# Finetune IRS : GPT-4

## Setup

In [ ]:
# Setup directory
%cd E:/Github_Repo/Info-Retrieve-AI/

In [ ]:
# Install required packages
%pip install -r raga_requirements.txt
%pip install sentence_transformers rouge nltk bert-score

In [ ]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Import necessary modules
import os
from __init__ import cfg
import pandas as pd
import requests
from bs4 import BeautifulSoup  # If web scraping is needed directly in RAGA
from huggingface_hub import HfApi  # For interacting with Hugging Face Hub if necessary
import ragas
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_relevancy, context_recall, context_entity_recall, answer_similarity, answer_correctness
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from pinecone import Pinecone, ServerlessSpec
import nltk
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
from datasets import Dataset
from bert_score import score as bert_score

# Set environment variables for API access
os.environ['HF_HOME'] = '/content/drive/MyDrive/Colab_Notebooks/Capstone_Project/Web_URL/IRS_Models/RAGA/huggingface'
os.environ['TRANSFORMERS_CACHE'] = os.environ['HF_HOME']
os.environ['HF_DATASETS_CACHE'] = os.environ['HF_HOME']
os.environ['HF_METRICS_CACHE'] = os.environ['HF_HOME']
os.environ['HF_TOKEN'] = cfg.HF_TOKEN
os.environ['OPENAI_API_KEY'] = cfg.OPENAI_API_KEY

nltk.download('punkt')
rouge = Rouge()

Mounted at /content/drive
/content/drive/MyDrive/Colab_Notebooks/Capstone_Project/Web_URL/IRS_Models/RAGA
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.5/214.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 7.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Load Synthetic Baseline Responses

In [ ]:
# Read the Dataset
data = pd.read_csv(r"E:\Github_Repo\Info-Retrieve-AI\RAGAs\output\syn_gpt_qa_50.csv")

# Fill NaN values in 'ground_truth' column with a specific placeholder
data['ground_truth'] = data['ground_truth'].fillna('No response available')
data

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What challenges do auto manufacturers face in ...,"['In both the UK and US, there is currently a ...",Severe price increases imposed via tariffs tha...,simple,[{'title': 'How Disruptive Pricing Will Impact...,True
1,What was the limitation of the quick-serve res...,[' quick-serve restaurant (QSR) we worked with...,The quick-serve restaurant's segmentation solu...,simple,"[{'title': 'Obtain a Deeper, Multidimensional ...",True
2,How does negative press coverage impact a bran...,[' spurred viral videos of fans destroying the...,No response available,simple,"[{'title': 'The Power, and Pitfalls, of Politi...",True
3,How does EVForward provide valuable insight in...,[' to much more effective results.\n\n\nEVForw...,EVForward provides valuable insight into the E...,simple,[{'title': 'Finding the Future of Mobility wit...,True
4,What challenges do landlords face when install...,"[' sharing (i.e., if multiple EVs are plugged ...",Landlords face a balancing act in installing i...,simple,[{'title': 'Multifamily Housing EV Charging: W...,True
5,How can effective sponsorship measurement help...,"[' marketers don’t see the value, or worse sti...",Effective sponsorship measurement provides det...,simple,[{'title': 'Measuring the Value of Sponsorship...,True
6,What is the current availability of charging s...,['EV Potential Not Yet Realized\n\n\nAs we loo...,The number of public charging stations has inc...,simple,"[{'title': 'The Challenges of Making EV Easy',...",True
7,How is the global rideshare industry projected...,['It’s safe to say that ridesharing services l...,The global rideshare industry is projected to ...,simple,[{'title': 'Rideshare Vehicles: A Meaningful B...,True
8,What is the importance of delivering multiface...,['there’s an obstacle in my way\n—\nperhaps I ...,The critical support offered to EV buyers need...,simple,[{'title': 'How to Create an “EV Easy” Mindset...,True
9,What are some of the specific areas that manuf...,"['For fans of the Terminator series of movies,...","Manufacturers, suppliers, and technology compa...",simple,[{'title': 'Rise of the Smart Car: The Shape-S...,True


## Load GPT4 responses based on synthetic baseline questions

In [ ]:
gpt4_responses = pd.read_csv(r"E:\Github_Repo\Info-Retrieve-AI\RAGAs\output\gpt4_syn_qa_responses.csv")
gpt4_responses

,question,contexts,ground_truth,evolution_type,metadata,episode_done,Response
0,What challenges do auto manufacturers face in ...,"['In both the UK and US, there is currently a ...",Severe price increases imposed via tariffs tha...,simple,[{'title': 'How Disruptive Pricing Will Impact...,True,Potential massive price hikes due to trade tar...
1,What was the limitation of the quick-serve res...,[' quick-serve restaurant (QSR) we worked with...,The quick-serve restaurant's segmentation solu...,simple,"[{'title': 'Obtain a Deeper, Multidimensional ...",True,The quick-serve restaurant's segmentation solu...
2,How does negative press coverage impact a bran...,[' spurred viral videos of fans destroying the...,No response available,simple,"[{'title': 'The Power, and Pitfalls, of Politi...",True,Negative press coverage can impact a brand's h...
3,How does EVForward provide valuable insight in...,[' to much more effective results.\n\n\nEVForw...,EVForward provides valuable insight into the E...,simple,[{'title': 'Finding the Future of Mobility wit...,True,EVForward provides valuable insight into the E...
4,What challenges do landlords face when install...,"[' sharing (i.e., if multiple EVs are plugged ...",Landlords face a balancing act in installing i...,simple,[{'title': 'Multifamily Housing EV Charging: W...,True,Landlords of multifamily housing face several ...
5,How can effective sponsorship measurement help...,"[' marketers don’t see the value, or worse sti...",Effective sponsorship measurement provides det...,simple,[{'title': 'Measuring the Value of Sponsorship...,True,Effective sponsorship measurement can help in ...
6,What is the current availability of charging s...,['EV Potential Not Yet Realized\n\n\nAs we loo...,The number of public charging stations has inc...,simple,"[{'title': 'The Challenges of Making EV Easy',...",True,The text does not provide a specific number or...
7,How is the global rideshare industry projected...,['It’s safe to say that ridesharing services l...,The global rideshare industry is projected to ...,simple,[{'title': 'Rideshare Vehicles: A Meaningful B...,True,The global rideshare industry is projected to ...
8,What is the importance of delivering multiface...,['there’s an obstacle in my way\n—\nperhaps I ...,The critical support offered to EV buyers need...,simple,[{'title': 'How to Create an “EV Easy” Mindset...,True,Delivering multifaceted EV support to EV buyer...
9,What are some of the specific areas that manuf...,"['For fans of the Terminator series of movies,...","Manufacturers, suppliers, and technology compa...",simple,[{'title': 'Rise of the Smart Car: The Shape-S...,True,"Manufacturers, suppliers, and technology compa..."


## RAGA Evaluator

In [ ]:
class LLMPerformanceEvaluator:
    def __init__(self, gpt4_responses):
        self.data = gpt4_responses
        self.rouge = Rouge()

    def preprocess_data(self):
        # Ensure ground truth data is present for all records to avoid processing errors
        filtered_data = self.data[self.data['ground_truth'] != 'No response available']

        # Tokenize text for metric calculation
        filtered_data['Reference_Tokens'] = filtered_data['ground_truth'].apply(lambda x: word_tokenize(x.lower()))
        filtered_data['Response_Tokens'] = filtered_data['Response'].apply(lambda x: word_tokenize(x.lower()))

        # Calculate BLEU and ROUGE scores only for entries with valid ground truth
        filtered_data['BLEU'] = filtered_data.apply(lambda row: sentence_bleu([row['Reference_Tokens']], row['Response_Tokens'], weights=(0.25, 0.25, 0.25, 0.25)), axis=1)
        filtered_data['ROGUE'] = filtered_data.apply(lambda row: rouge.get_scores(row['Response'], row['ground_truth'])[0]['rouge-l']['f'], axis=1)

        # Calculate BERT scores
        references = filtered_data['ground_truth'].tolist()
        predictions = filtered_data['Response'].tolist()
        P, R, F1 = bert_score(predictions, references, lang="en", verbose=True)
        filtered_data['BERT_P'] = P.numpy()
        filtered_data['BERT_R'] = R.numpy()
        filtered_data['BERT_F1'] = F1.numpy()

        # Prepare data for RAGA metrics
        self.dataset = Dataset.from_dict({
            'question': filtered_data['question'].tolist(),
            'answer': filtered_data['Response'].tolist(),
            'contexts': [nltk.sent_tokenize(x) for x in filtered_data['ground_truth']],
            'ground_truth': filtered_data['ground_truth'].tolist()
        })

        self.filtered_data = filtered_data
        return self.filtered_data

        # dataset = Dataset.from_dict(data_samples)

    def compute_raga_metrics(self, metrics):
        # Compute RAGA metrics in batches
        raga_results = evaluate(self.dataset, metrics=metrics)
        raga_df = raga_results.to_pandas()
        return raga_df

    def compute_metrics(self):
        self.preprocess_data()
        # Define batches of metrics
        metric_batches = [
            [faithfulness, answer_relevancy, context_precision],
            [context_relevancy, context_recall, context_entity_recall],
            [answer_similarity, answer_correctness]
        ]
        df = [self.filtered_data]

        for metrics in metric_batches:
            metrics_result = self.compute_raga_metrics(metrics)
            df.append(metrics_result)
        combine_df = pd.concat(df, axis=1)
        return combine_df


## Evaluate Performance Metrics for GPT-4

In [ ]:
evaluator = LLMPerformanceEvaluator(gpt4_responses)

In [ ]:
evaluator.preprocess_data()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 364.69 seconds, 0.13 sentences/sec


,question,contexts,ground_truth,evolution_type,metadata,episode_done,Response,Reference_Tokens,Response_Tokens,BLEU,ROGUE,BERT_P,BERT_R,BERT_F1
0,What challenges do auto manufacturers face in ...,"['In both the UK and US, there is currently a ...",Severe price increases imposed via tariffs tha...,simple,[{'title': 'How Disruptive Pricing Will Impact...,True,Potential massive price hikes due to trade tar...,"[severe, price, increases, imposed, via, tarif...","[potential, massive, price, hikes, due, to, tr...",9.499367e-02,0.321678,0.874894,0.910280,0.892236
1,What was the limitation of the quick-serve res...,[' quick-serve restaurant (QSR) we worked with...,The quick-serve restaurant's segmentation solu...,simple,"[{'title': 'Obtain a Deeper, Multidimensional ...",True,The quick-serve restaurant's segmentation solu...,"[the, quick-serve, restaurant, 's, segmentatio...","[the, quick-serve, restaurant, 's, segmentatio...",2.985456e-01,0.571429,0.921405,0.971298,0.945694
3,How does EVForward provide valuable insight in...,[' to much more effective results.\n\n\nEVForw...,EVForward provides valuable insight into the E...,simple,[{'title': 'Finding the Future of Mobility wit...,True,EVForward provides valuable insight into the E...,"[evforward, provides, valuable, insight, into,...","[evforward, provides, valuable, insight, into,...",1.173411e-01,0.364780,0.862064,0.924760,0.892312
4,What challenges do landlords face when install...,"[' sharing (i.e., if multiple EVs are plugged ...",Landlords face a balancing act in installing i...,simple,[{'title': 'Multifamily Housing EV Charging: W...,True,Landlords of multifamily housing face several ...,"[landlords, face, a, balancing, act, in, insta...","[landlords, of, multifamily, housing, face, se...",1.488835e-01,0.336957,0.870464,0.911350,0.890438
5,How can effective sponsorship measurement help...,"[' marketers don’t see the value, or worse sti...",Effective sponsorship measurement provides det...,simple,[{'title': 'Measuring the Value of Sponsorship...,True,Effective sponsorship measurement can help in ...,"[effective, sponsorship, measurement, provides...","[effective, sponsorship, measurement, can, hel...",1.492871e-01,0.431373,0.907173,0.941681,0.924105
6,What is the current availability of charging s...,['EV Potential Not Yet Realized\n\n\nAs we loo...,The number of public charging stations has inc...,simple,"[{'title': 'The Challenges of Making EV Easy',...",True,The text does not provide a specific number or...,"[the, number, of, public, charging, stations, ...","[the, text, does, not, provide, a, specific, n...",3.010855e-155,0.202247,0.867870,0.879603,0.873697
7,How is the global rideshare industry projected...,['It’s safe to say that ridesharing services l...,The global rideshare industry is projected to ...,simple,[{'title': 'Rideshare Vehicles: A Meaningful B...,True,The global rideshare industry is projected to ...,"[the, global, rideshare, industry, is, project...","[the, global, rideshare, industry, is, project...",1.000000e+00,1.000000,1.000000,1.000000,1.000000
8,What is the importance of delivering multiface...,['there’s an obstacle in my way\n—\nperhaps I ...,The critical support offered to EV buyers need...,simple,[{'title': 'How to Create an “EV Easy” Mindset...,True,Delivering multifaceted EV support to EV buyer...,"[the, critical, support, offered, to, ev, buye...","[delivering, multifaceted, ev, support, to, ev...",9.244919e-02,0.311927,0.866447,0.912400,0.888830
9,What are some of the specific areas that manuf...,"['For fans of the Terminator series of movies,...","Manufacturers, suppliers, and technology compa...",simple,[{'title': 'Rise of the Smart Car: The Shape-S...,True,"Manufacturers, suppliers, and technology compa...","[manufacturers, ,, suppliers, ,, and, technolo...","[manufacturers, ,, suppliers, ,, and, technolo...",2.863862e-02,0.094862,0.806888,0.908188,0.854546
10,What are some factors that can spur the adopti...,['asm? Fleet decision-makers want

In [ ]:
metrics_df = evaluator.compute_metrics()

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 351.84 seconds, 0.13 sentences/sec


Evaluating:   0%|          | 0/138 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/138 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/92 [00:00<?, ?it/s]

In [ ]:
pd.set_option('display.max_columns', None)
metrics_df

,Response_Tokens,BERT_R,answer,Reference_Tokens,answer_correctness,answer_similarity,evolution_type,faithfulness,metadata,BERT_F1,BLEU,context_recall,context_entity_recall,contexts,ground_truth,question,Response,ROGUE,answer_relevancy,context_relevancy,BERT_P,context_precision,episode_done


In [ ]:
column_list = metrics_df.columns
print(column_list)

Index(['Response_Tokens', 'BERT_R', 'answer', 'Reference_Tokens',
       'answer_correctness', 'answer_similarity', 'evolution_type',
       'faithfulness', 'metadata', 'BERT_F1', 'BLEU', 'context_recall',
       'context_entity_recall', 'contexts', 'ground_truth', 'question',
       'Response', 'ROGUE', 'answer_relevancy', 'context_relevancy', 'BERT_P',
       'context_precision', 'episode_done'],
      dtype='object')


In [ ]:
metrics_df

,Response_Tokens,BERT_R,answer,Reference_Tokens,answer_correctness,answer_similarity,evolution_type,faithfulness,metadata,BERT_F1,BLEU,context_recall,context_entity_recall,contexts,ground_truth,question,Response,ROGUE,answer_relevancy,context_relevancy,BERT_P,context_precision,episode_done


In [ ]:
column_list = set(column_list)
column_list

{'BERT_F1',
 'BERT_P',
 'BERT_R',
 'BLEU',
 'ROGUE',
 'Reference_Tokens',
 'Response',
 'Response_Tokens',
 'answer',
 'answer_correctness',
 'answer_relevancy',
 'answer_similarity',
 'context_entity_recall',
 'context_precision',
 'context_recall',
 'context_relevancy',
 'contexts',
 'episode_done',
 'evolution_type',
 'faithfulness',
 'ground_truth',
 'metadata',
 'question'}

In [ ]:
final_df = pd.DataFrame(metrics_df,columns=list(column_list))

In [ ]:
final_df.columns

Index(['Response_Tokens', 'BERT_R', 'answer', 'Reference_Tokens',
       'answer_correctness', 'answer_similarity', 'evolution_type',
       'faithfulness', 'metadata', 'BERT_F1', 'BLEU', 'context_recall',
       'context_entity_recall', 'contexts', 'ground_truth', 'question',
       'Response', 'ROGUE', 'answer_relevancy', 'context_relevancy', 'BERT_P',
       'context_precision', 'episode_done'],
      dtype='object')

In [ ]:
metrics_df

,Response_Tokens,BERT_R,answer,answer,answer,Reference_Tokens,answer_correctness,answer_similarity,evolution_type,faithfulness,metadata,BERT_F1,BLEU,context_recall,context_entity_recall,contexts,contexts,contexts,contexts,ground_truth,ground_truth,ground_truth,ground_truth,question,question,question,question,Response,ROGUE,answer_relevancy,context_relevancy,BERT_P,context_precision,episode_done


In [ ]:
pd.set_option('display.max_columns', None)
metrics_df.to_csv(r"E:\Github_Repo\Info-Retrieve-AI\RAGAs\output\gpt4_performance_metrics.csv", index=False)
print("Performance metrics have been saved successfully.")

In [ ]:
metrics_df.head()

## Test

In [ ]:
class LLMPerformanceEvaluator:

    def __init__(self, data):
        # self.model = model
        # self.indexer = indexer
        self.data = gpt4_responses

    def compute_metrics(self):
        # Ensure ground truth data is present for all records to avoid processing errors
        filtered_data = self.data.dropna(subset=['ground_truth'])

        # Preprocessing text for metric calculation
        filtered_data['Reference_Tokens'] = filtered_data['ground_truth'].apply(lambda x: nltk.word_tokenize(x.lower()))
        filtered_data['Response_Tokens'] = filtered_data['Response'].apply(lambda x: nltk.word_tokenize(x.lower()))

        # Calculate BLEU and ROUGE scores only for entries with valid ground truth
        filtered_data['BLEU'] = filtered_data.apply(lambda row: sentence_bleu([row['Reference_Tokens']], row['Response_Tokens'], weights=(0.25, 0.25, 0.25, 0.25)), axis=1)
        filtered_data['ROGUE'] = filtered_data.apply(lambda row: rouge.get_scores(row['Response'], row['ground_truth'])[0]['rouge-l']['f'], axis=1)


        # Calculate BERT scores
        # Convert to list for BERT score calculation
        references = filtered_data['ground_truth'].tolist()
        predictions = filtered_data['Response'].tolist()

        # Calculate BERT scores
        P, R, F1 = bert_score(predictions, references, lang="en", verbose=True)

        # Add BERT scores to DataFrame
        filtered_data['BERT_P'] = P.numpy()
        filtered_data['BERT_R'] = R.numpy()
        filtered_data['BERT_F1'] = F1.numpy()

        # Prepare data samples for RAGA metrics, ensuring all necessary fields are present
        data_samples = {
            'question': filtered_data['question'].tolist(),
            'answer': filtered_data['Response'].tolist(),
            'contexts': [nltk.sent_tokenize(x) for x in filtered_data['ground_truth']],
            'ground_truth': filtered_data['ground_truth'].tolist()  # Ensuring ground_truth is included for RAGA metrics
        }
        dataset = Dataset.from_dict(data_samples)

        # # Compute RAGA metrics
        # raga_results = evaluate(dataset, metrics=[
        #     faithfulness,
        #     # answer_relevancy, context_precision, context_relevancy,
        #     # context_recall, context_entity_recall,
        #     answer_similarity, answer_correctness
        # ])
        # raga_df = raga_results.to_pandas()

        # # Combine all metrics into a single DataFrame
        # self.metrics_df = pd.concat([filtered_data, raga_df], axis=1)
        # return self.metrics_df

    def get_metrices_from_raga(self, metrices_param):
      # Compute RAGA metrics
      raga_results = evaluate(self.dataset, metrics= metrices_param)
      raga_df = raga_results.to_pandas()
      # Combine all metrics into a single DataFrame
      metrics_df = pd.concat([self.filtered_data, raga_df], axis=1)
      return metrics_df

In [ ]:
metrics_dict = {
    1: 'faithfulness',
    2: 'answer_relevancy',
    3: 'context_precision',
    4: 'context_relevancy',
    5: 'context_recall',
    6: 'context_entity_recall',
    7: 'answer_similarity',
    8: 'answer_correctness'
}

metrics_dict[1]

In [ ]:
# metrics_df.to_csv('/path/to/output/GPT-4_Performance_Metrics.csv', index=False)
evaluator.metrics_df.to_csv('/content/drive/MyDrive/Colab_Notebooks/Capstone_Project/Web_URL/IRS_Models/RAGA/GPT-4_Performance_Metrics.csv', index=False)
print("Performance metrics have been saved successfully.")
print(metrics_df.head())

In [ ]:
evaluator = LLMPerformanceEvaluator(data)

In [ ]:
evaluator.compute_metrics()

In [ ]:
metrices_param = []
for item in metrics_dict.values():
  metrices_param.append(item)
  evaluator.get_metrices_from_raga(metrices_param)

In [ ]:
evaluator.metrics_df.to_csv('/content/drive/MyDrive/Colab_Notebooks/Capstone_Project/Web_URL/IRS_Models/RAGA/GPT-4_Performance_Metrics.csv', index=False)
print("Performance metrics have been saved successfully.")
# print(evaluator.metrics_df.head())